## Handling Missing Data: MCAR, MAR, MNAR and MICE



### 🚩 Types of Missingness

#### 1️⃣ MCAR (Missing Completely At Random)
- Definition: Missingness is unrelated to any variable (observed or unobserved).  
- Example: A survey response lost in transit.

#### 2️⃣ MAR (Missing At Random)
- Definition: Missingness can be explained _by other observed variables_, but **not** by the missing value itself.
- Example: Income is missing more for younger people (if age is observed, missingness is MAR).

#### 3️⃣ MNAR (Missing Not At Random)
- Definition: Missingness _depends on unobserved data_ (often the missing value itself).
- Example: High earners don’t report their income (and income itself determines whether it’s missing).



### 🧩 MICE (Multiple Imputation by Chained Equations)

- Assumption: Data is **MAR** (Missing At Random).

- Advantage:  
    - Can provide **quite accurate estimates** because it preserves relationships between variables.
    - Produces **multiple imputed datasets** to reflect statistical uncertainty.
- Disadvantages: 
    - **Computationally slow** (especially with large data or lots of iterations).
    - Requires **all training data on the server** since imputations are conditional on all variables.




### How Mice Works

Let's Assume, we have a startup data


|      | R&D Spend | Administration | Marketing Spend | Profit |
|------|-----------|---------------|----------------|--------|
| 21   | 8.0       | 15.0          | 30.0           | 11.0   |
| 37   | 4.0       | 5.0           | 20.0           | 9.0    |
| 2    | 15.0      | 10.0          | 41.0           | 19.0   |
| 14   | 12.0      | 16.0          | 26.0           | 13.0   |
| 44   | 2.0       | 15.0          | 3.0            | 7.0    |


We will remove the target column


|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|---------------|----------------|
| 21   | 8.0       | 15.0          | 30.0           |
| 37   | 4.0       | 5.0           | 20.0           |
| 2    | 15.0      | 10.0          | 41.0           |
| 14   | 12.0      | 16.0          | 26.0           |
| 44   | 2.0       | 15.0          | 3.0            |

Assume we have some NaN values, which we want to impute with MICE

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|---------------|----------------|
| 21   | 8.0       | 15.0          | 30.0           |
| 37   | NaN       | 5.0           | 20.0           |
| 2    | 15.0      | 10.0          | 41.0           |
| 14   | 12.0      | NaN           | 26.0           |
| 44   | 2.0       | 15.0          | NaN            |

Step 1: Fill all NaN with mean of respective cols

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|---------------|----------------|
| 21   | 8.00      | 15.00         | 30.00          |
| 37   | 9.25      | 5.00          | 20.00          |
| 2    | 15.00     | 10.00         | 41.00          |
| 14   | 12.00     | 11.25         | 26.00          |
| 44   | 2.00      | 15.00         | 29.25          |


- now what you do is remove that row with NaN value and treat that row (leftmost column) as test data.
- And then train any linear model on reamining and then predict that NaN value
- And then repeat this to all NaN values you replaced 

Step 2: Now what you do is remove the mean value which you relaced with NaN, and put NaN value again (only in first left column)


|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   |   8.0     |     15.00      |     30.00      |
| 37   |    NaN    |      5.00      |     20.00      |
| 2    |  15.0     |     10.00      |     41.00      |
| 14   |  12.0     |     11.25      |     26.00      |
| 44   |   2.0     |     15.00      |     29.25      |

Step 3: Now predict the value of NaN by training any linear model on your data

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   |   8.0     |     15.00      |     30.00      |
| 37   |    23.14    |      5.00      |     20.00      |
| 2    |  15.0     |     10.00      |     41.00      |
| 14   |  12.0     |     11.25      |     26.00      |
| 44   |   2.0     |     15.00      |     29.25      |

Step 4: Now repeat the the process on second left column, remove and replace all the values you replaced with mean back to NaN 

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   |   8.0     |     15.00      |     30.00      |
| 37   |    23.14    |      5.00      |     20.00      |
| 2    |  15.0     |     10.00      |     41.00      |
| 14   |  12.0     |     NaN      |     26.00      |
| 44   |   2.0     |     15.00      |     29.25      |

Step 5: Predict the missing values of coloumn 2 using other columns

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   |   8.0     |     15.00      |     30.00      |
| 37   |    23.14    |      5.00      |     20.00      |
| 2    |  15.0     |     10.00      |     41.00      |
| 14   |  12.0     |     11.06      |     26.00      |
| 44   |   2.0     |     15.00      |     29.25      |

Step 6: Do the same with column 3, and all other column if have more

- Now if have to find the difference between the table repleced with mean and the table which was predicted by model
- And if the difference of the missing values is 0 then you are good the table with model prediction is final
- But if the difference is not 0 then do the exact same process of taking mean replacing the missing value with mean and all the process




Iteration 0

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 8.00      | 15.00          | 30.00          |
| 37   | 9.25      | 5.00           | 20.00          |
| 2    | 15.00     | 10.00          | 41.00          |
| 14   | 12.00     | 11.25          | 26.00          |
| 44   | 2.00      | 15.00          | 29.25          |

Iteration 1

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 8.00      | 15.00          | 30.00          |
| 37   | 23.14     | 5.00           | 20.00          |
| 2    | 15.00     | 10.00          | 41.00          |
| 14   | 12.00     | 11.06          | 26.00          |
| 44   | 2.00      | 15.00          | 31.56          |

Difference

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 0.00      | 0.00           | 0.00           |
| 37   | 13.89     | 0.00           | 0.00           |
| 2    | 0.00      | 0.00           | 0.00           |
| 14   | 0.00      | -0.19          | 0.00           |
| 44   | 0.00      | 0.00           | 2.31           |



Second round

Iteration 1

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 8.00      | 15.00          | 30.00          |
| 37   | 23.14     | 5.00           | 20.00          |
| 2    | 15.00     | 10.00          | 41.00          |
| 14   | 12.00     | 11.06          | 26.00          |
| 44   | 2.00      | 15.00          | 31.56          |

Iteration 2

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 8.00      | 15.00          | 30.00          |
| 37   | 23.78     | 5.00           | 20.00          |
| 2    | 15.00     | 10.00          | 41.00          |
| 14   | 12.00     | 11.22          | 26.00          |
| 44   | 2.00      | 15.00          | 31.56          |

Difference

|      | R&D Spend | Administration | Marketing Spend |
|------|-----------|----------------|----------------|
| 21   | 0.00      | 0.00           | 0.00           |
| 37   | 0.64      | 0.00           | 0.00           |
| 2    | 0.00      | 0.00           | 0.00           |
| 14   | 0.00      | 0.16           | 0.00           |
| 44   | 0.00      | 0.00           | 0.00           |



In [45]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

In [49]:
df = np.round(pd.read_csv('50_Startups.csv')[['R&D Spend','Administration','Marketing Spend','Profit']]/10000)
np.random.seed(9)
df = df.sample(5)
df

,R&D Spend,Administration,Marketing Spend,Profit
21,8.0,15.0,30.0,11.0
37,4.0,5.0,20.0,9.0
2,15.0,10.0,41.0,19.0
14,12.0,16.0,26.0,13.0
44,2.0,15.0,3.0,7.0


In [50]:
df = df.iloc[:, 0:-1]
df.head()

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,4.0,5.0,20.0
2,15.0,10.0,41.0
14,12.0,16.0,26.0
44,2.0,15.0,3.0


In [51]:
df.iloc[1,0] = np.nan
df.iloc[3,1] = np.nan
df.iloc[-1,-1] = np.nan

C:\Users\premk\AppData\Local\Temp\ipykernel_12756\766688332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[1,0] = np.nan
C:\Users\premk\AppData\Local\Temp\ipykernel_12756\766688332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[3,1] = np.nan
C:\Users\premk\AppData\Local\Temp\ipykernel_12756\766688332.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1,-1] = np.nan


In [52]:
df.head()

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,NaN,5.0,20.0
2,15.0,10.0,41.0
14,12.0,NaN,26.0
44,2.0,15.0,NaN


In [53]:
# Step 1 - Impute all missing values with mean of respective col

df0 = pd.DataFrame()

df0['R&D Spend'] = df['R&D Spend'].fillna(df['R&D Spend'].mean())
df0['Administration'] = df['Administration'].fillna(df['Administration'].mean())
df0['Marketing Spend'] = df['Marketing Spend'].fillna(df['Marketing Spend'].mean())

In [54]:
df0.head()

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,9.25,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


In [56]:
# Remove the col1 imputed value
df1 = df0.copy()

df1.iloc[1,0] = np.nan

df1

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.25,26.00
44,2.0,15.00,29.25


In [57]:
# Use first 3 rows to build a model and use the last for prediction

X = df1.iloc[[0,2,3,4],1:3]
X

,Administration,Marketing Spend
21,15.00,30.00
2,10.00,41.00
14,11.25,26.00
44,15.00,29.25


In [58]:
y = df1.iloc[[0,2,3,4],0]
y

21     8.0
2     15.0
14    12.0
44     2.0
Name: R&D Spend, dtype: float64

In [59]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[1,1:].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([23.14158651])

In [60]:
df1.iloc[1,0] = 23.14
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


In [62]:
# Remove the col2 imputed value

df1.iloc[3,1] = np.nan

df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.0,30.00
37,23.14,5.0,20.00
2,15.00,10.0,41.00
14,12.00,NaN,26.00
44,2.00,15.0,29.25


In [63]:
# Use last 3 rows to build a model and use the first for prediction
X = df1.iloc[[0,1,2,4],[0,2]]
X

,R&D Spend,Marketing Spend
21,8.00,30.00
37,23.14,20.00
2,15.00,41.00
44,2.00,29.25


In [64]:
y = df1.iloc[[0,1,2,4],1]
y

21    15.0
37     5.0
2     10.0
44    15.0
Name: Administration, dtype: float64

In [65]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[3,[0,2]].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.06331285])

In [66]:
df1.iloc[3,1] = 11.06
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,29.25


In [68]:
# Remove the col3 imputed value
df1.iloc[4,-1] = np.nan

df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.0
37,23.14,5.00,20.0
2,15.00,10.00,41.0
14,12.00,11.06,26.0
44,2.00,15.00,NaN


In [69]:
# Use last 3 rows to build a model and use the first for prediction
X = df1.iloc[0:4,0:2]
X

,R&D Spend,Administration
21,8.00,15.00
37,23.14,5.00
2,15.00,10.00
14,12.00,11.06


In [70]:
y = df1.iloc[0:4,-1]
y

21    30.0
37    20.0
2     41.0
14    26.0
Name: Marketing Spend, dtype: float64

In [71]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[4,0:2].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([31.56351448])

In [72]:
df1.iloc[4,-1] = 31.56
# After 1st Iteration
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


In [73]:
# Subtract 0th iteration from 1st iteration

df1 - df0

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,13.89,0.00,0.00
2,0.00,0.00,0.00
14,0.00,-0.19,0.00
44,0.00,0.00,2.31


In [75]:
df2 = df1.copy()

df2.iloc[1,0] = np.nan

df2

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.06,26.00
44,2.0,15.00,31.56


In [76]:
X = df2.iloc[[0,2,3,4],1:3]
y = df2.iloc[[0,2,3,4],0]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[1,1:].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([23.78627207])

In [77]:
df2.iloc[1,0] = 23.78

In [79]:
df2.iloc[3,1] = np.nan
X = df2.iloc[[0,1,2,4],[0,2]]
y = df2.iloc[[0,1,2,4],1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[3,[0,2]].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.22020174])

In [80]:
df2.iloc[3,1] = 11.22

In [82]:
df2.iloc[4,-1] = np.nan

X = df2.iloc[0:4,0:2]
y = df2.iloc[0:4,-1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df2.iloc[4,0:2].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([38.87979054])

In [83]:
df2.iloc[4,-1] = 31.56
df2

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.78,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.22,26.00
44,2.00,15.00,31.56


In [84]:
df2 - df1

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.0
37,0.64,0.00,0.0
2,0.00,0.00,0.0
14,0.00,0.16,0.0
44,0.00,0.00,0.0


In [86]:
df3 = df2.copy()

df3.iloc[1,0] = np.nan

df3

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.22,26.00
44,2.0,15.00,31.56


In [87]:
X = df3.iloc[[0,2,3,4],1:3]
y = df3.iloc[[0,2,3,4],0]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[1,1:].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([24.57698058])

In [88]:
df3.iloc[1,0] = 24.57

In [90]:
df3.iloc[3,1] = np.nan
X = df3.iloc[[0,1,2,4],[0,2]]
y = df3.iloc[[0,1,2,4],1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[3,[0,2]].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([11.37282844])

In [91]:
df3.iloc[3,1] = 11.37

In [93]:
df3.iloc[4,-1] = np.nan

X = df3.iloc[0:4,0:2]
y = df3.iloc[0:4,-1]

lr = LinearRegression()
lr.fit(X,y)
lr.predict(df3.iloc[4,0:2].values.reshape(1,2))

e:\50-days-of-ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([45.53976417])

In [94]:
df3.iloc[4,-1] = 45.53
df2.iloc[3,1] = 11.22
df3

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,24.57,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.37,26.00
44,2.00,15.00,45.53


In [95]:
df3 - df2

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,0.79,0.00,0.00
2,0.00,0.00,0.00
14,0.00,0.15,0.00
44,0.00,0.00,13.97
